XGBoost ~ gridcv tuning

# Functions

## transform_features & ~2

In [1]:
"""
Pre-processing functions
Add functions you need
"""

## Functions
from sklearn import preprocessing
import pandas as pd


def drop_features(df):
    df.drop(['PassengerId','Name','Ticket'], axis=1, inplace=True)
    return df


def fillna(df):
    Age_mean = df['Age'].mean()
    Age_mode = df['Age'].mode()
    freq = df['Embarked'].value_counts().idxmax()
    
    df['Cabin'].fillna('N', inplace=True)
    df['Cabin'] = df['Cabin'].str[:1]  #indexing
    df['Age'].fillna(Age_mean, inplace=True)
    df['Embarked'].fillna(freq, inplace=True)
    return df


def fillna_2(df):
    Age_mean = df['Age'].mean()
    Age_mode = df['Age'].mode()
    freq = df['Embarked'].value_counts().idxmax()
    Fare_mean = df['Fare'].mean()
    
    df['Cabin'].fillna('N', inplace=True)
    df['Cabin'] = df['Cabin'].str[:1]  
    df['Fare'].fillna(Fare_mean, inplace=True)
    df['Age'].fillna(Age_mean, inplace=True)
    df['Embarked'].fillna(freq, inplace=True)
    return df



## For use
def transform_features(df):
    df = drop_features(df)
    df = fillna(df)
    df = pd.get_dummies(df, drop_first=True)
    return df


def transform_features_2(df):
    df = drop_features(df)
    df = fillna_2(df)
    df = pd.get_dummies(df, drop_first=True)
    return df

## get_outlier & transform_outlier

In [2]:
def get_outlier(df=None, column=None, weight=1.5):
    #Object
    var1 = df[column]
    
    #25%, 75%  #numpy value ---> percentage
    quantile_25 = np.percentile(var1.values,25)
    quantile_75 = np.percentile(var1.values,75)
    
    #iqr
    iqr = quantile_75 - quantile_25
    iqr_weight = iqr *weight
    lowest_val = quantile_25 - iqr_weight
    highest_val = quantile_75 + iqr_weight
    
    #conditions
    outlier_index = var1[(var1<lowest_val)|(var1>highest_val)].index
    #outlier_index = var1[(var1<lowest_val)].index
    #outlier_index = var1[(var1>highest_val)].index
    
    print("<Outliers>")
    print("highest_val :", highest_val)
    print("lowest_val :", lowest_val)
    print("len :",len(outlier_index), end='\n\n')    
    return outlier_index, highest_val, lowest_val


def transform_outlier(df=None, column=None, weight=1.5):
    x, var1, var2 = get_outlier(df=df, column=column, weight=weight)
    #df[column].iloc[x] = x  #both
    df[column].iloc[x] = var1  #highest index
    #df[column].iloc[x] = var2  #lowest index
    return df

# Set Trainset 

In [3]:
# module import 
from sklearn import metrics
import numpy as np
import pandas as pd
import seaborn as sns

# Ignore warnings
import warnings 
warnings.filterwarnings('ignore')

# data load 
df = pd.read_csv("C:\\Users\\icecr\\OneDrive\\CSV_file\\titanic\\train.csv")
#df = pd.read_csv("C:\\Users\\82103\\OneDrive\\CSV_file\\titanic\\train.csv")  #for notebook

# Devide feature & label
y_df = df['Survived']
X_df = df.drop('Survived', axis=1)

# Apply pre-processing func
X_df = transform_features(X_df)
X_df = transform_outlier(df=X_df, column='Fare', weight=4.5)

X_df.info()

#pd.set_option('display.max_rows',500)  #row 출력개수 설정(자동생략 option off 기능)
#X_df.head(50)

<Outliers>
highest_val : 134.9032
lowest_val : -95.9928
len : 34

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Pclass      891 non-null    int64  
 1   Age         891 non-null    float64
 2   SibSp       891 non-null    int64  
 3   Parch       891 non-null    int64  
 4   Fare        891 non-null    float64
 5   Sex_male    891 non-null    uint8  
 6   Cabin_B     891 non-null    uint8  
 7   Cabin_C     891 non-null    uint8  
 8   Cabin_D     891 non-null    uint8  
 9   Cabin_E     891 non-null    uint8  
 10  Cabin_F     891 non-null    uint8  
 11  Cabin_G     891 non-null    uint8  
 12  Cabin_N     891 non-null    uint8  
 13  Cabin_T     891 non-null    uint8  
 14  Embarked_Q  891 non-null    uint8  
 15  Embarked_S  891 non-null    uint8  
dtypes: float64(2), int64(3), uint8(11)
memory usage: 44.5 KB


In [4]:
# delete Cabin_T ---> testset don't has Cabin_T, trainset has 1 data
X_df = X_df.drop(['Cabin_T'], axis=1)
X_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Pclass      891 non-null    int64  
 1   Age         891 non-null    float64
 2   SibSp       891 non-null    int64  
 3   Parch       891 non-null    int64  
 4   Fare        891 non-null    float64
 5   Sex_male    891 non-null    uint8  
 6   Cabin_B     891 non-null    uint8  
 7   Cabin_C     891 non-null    uint8  
 8   Cabin_D     891 non-null    uint8  
 9   Cabin_E     891 non-null    uint8  
 10  Cabin_F     891 non-null    uint8  
 11  Cabin_G     891 non-null    uint8  
 12  Cabin_N     891 non-null    uint8  
 13  Embarked_Q  891 non-null    uint8  
 14  Embarked_S  891 non-null    uint8  
dtypes: float64(2), int64(3), uint8(10)
memory usage: 43.6 KB


# Testing:trainset
## without tuning tool

In [5]:
# Devide trainset:testset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=0)

# Verify label ratio 
train_cnt = y_train.count()
test_cnt = y_test.count()

print('trainset :', X_train.shape)
print('testset :',X_test.shape, end='\n\n')

print("trainset label ratio")
print(y_train.value_counts()/train_cnt, end='\n\n')

print("testset label ratio")
print(y_test.value_counts()/test_cnt)

trainset : (712, 15)
testset : (179, 15)

trainset label ratio
0    0.616573
1    0.383427
Name: Survived, dtype: float64

testset label ratio
0    0.614525
1    0.385475
Name: Survived, dtype: float64


In [6]:
## Models

# Import module
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score

# Make object : classifier
dt_clf = DecisionTreeClassifier()
rf_clf = RandomForestClassifier()
lr_clf = LogisticRegression()


#DecisionTree
dt_clf.fit(X_train, y_train)
dt_pred = dt_clf.predict(X_test)
print('dt_Acc : {0:.4f}'.format(accuracy_score(y_test, dt_pred)))
print('dt_roc_auc : {0:.4f}'.format(roc_auc_score(y_test, dt_clf.predict_proba(X_test)[:,1])))
print('')


#RandomForest
rf_clf.fit(X_train, y_train)
rf_pred = rf_clf.predict(X_test)
print('rf_Acc : {0:.4f}'.format(accuracy_score(y_test, rf_pred)))
print('rf_roc_auc : {0:.4f}'.format(roc_auc_score(y_test, rf_clf.predict_proba(X_test)[:,1])))
print('')


#LogisticRegression
lr_clf.fit(X_train, y_train)
lr_pred = lr_clf.predict(X_test)
print('lr_Acc : {0:.4f}'.format(accuracy_score(y_test, lr_pred)))
print('lr_roc_auc : {0:.4f}'.format(roc_auc_score(y_test, lr_clf.predict_proba(X_test)[:,1])))
print('')


##XGBoost
from xgboost import XGBClassifier
xgb_clf = XGBClassifier(n_estimators=100)
xgb_clf.fit(X_train, y_train)
           
xgb_roc_score = roc_auc_score(y_test, xgb_clf.predict_proba(X_test)[:, 1])
xgb_acc_score = accuracy_score(y_test, xgb_clf.predict(X_test))
print('')
print('xgb_Acc : {0:.4f}'.format(xgb_acc_score))
print('xgb_roc_auc : {0:.4f}'.format(xgb_roc_score))

dt_Acc : 0.7821
dt_roc_auc : 0.7580

rf_Acc : 0.8101
rf_roc_auc : 0.8870

lr_Acc : 0.8156
lr_roc_auc : 0.8689

[20:47:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.

xgb_Acc : 0.8547
xgb_roc_auc : 0.8709


## GridCV : XGBoost

In [7]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score


params = {#general params
          'booster':['gbtree'],  #default=gbtree, other:gblinear
          'silent':[0],  #default=0, other:1(=silent)    
          
          #Booster params
          'learning_rate':[0.1, 0.05],  #default=0.1
          'n_estimators':[100, 1000],  #default=100
          'min_child_weight':[1],  #default=1
          'min_split_loss':[0],  #default=0  #gamma
          'max_depth':[3],  #default=3
          'subsample':[1],  #default=1
          'colsample_bytree':[1],  #default=1
          'reg_lamda':[1],  #default=1
          'reg_alpha':[0],  #default=0
          'scale_pos_weight':[1],  #default=1
    
          #Training task params    
          'objective':['binary:logistic'],  #default=logistic, other:index top
          'eval_metric':['error']  #default=rmse|error  #ohter : mae, logloss, merror, mlogloss
          }


xgb_clf = XGBClassifier()

xgb_grid = GridSearchCV(xgb_clf, 
                        param_grid=params, 
                        cv=5,
                        scoring='accuracy',  #default=accuracy  #options : roc_auc, f1
                        refit=True,
                        n_jobs=-1, 
                        verbose=1
                       )

xgb_grid.fit(X_df, y_df)


## [Early stopping code]
# gridcv.fit(X_train, y_train, early_stopping_rounds=100, eval_metric='auc',
#           eval_set=[(X_train, y_train), (X_test, y_test)])

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  20 | elapsed:    6.5s remaining:    6.5s


[20:47:17] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { reg_lamda, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:   14.2s finished


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs...
             n_jobs=-1,
             param_grid={'booster': ['gbtree'], 'colsample_bytree': [1],
                         'eval_metric': ['error'], 'learning_rate': [0.1, 0.05],
                         'max_depth': [3], 'min_child_weight': [1],
                         'min_split_l

In [8]:
## Result
print('Best score :', xgb_grid.best_score_, end='\n\n')
print('Best parameters :')
xgb_grid.best_params_

Best score : 0.8260686711443098

Best parameters :


{'booster': 'gbtree',
 'colsample_bytree': 1,
 'eval_metric': 'error',
 'learning_rate': 0.1,
 'max_depth': 3,
 'min_child_weight': 1,
 'min_split_loss': 0,
 'n_estimators': 100,
 'objective': 'binary:logistic',
 'reg_alpha': 0,
 'reg_lamda': 1,
 'scale_pos_weight': 1,
 'silent': 0,
 'subsample': 1}

In [9]:
# Result details
cv_res_df = pd.DataFrame(xgb_grid.cv_results_)
cv_res_df.sort_values(by=['rank_test_score'], inplace=True)
cv_res_df[['params', 'mean_test_score', 'rank_test_score']].head()

,params,mean_test_score,rank_test_score
0,"{'booster': 'gbtree', 'colsample_bytree': 1, '...",0.826069,1
2,"{'booster': 'gbtree', 'colsample_bytree': 1, '...",0.820444,2
3,"{'booster': 'gbtree', 'colsample_bytree': 1, '...",0.812604,3
1,"{'booster': 'gbtree', 'colsample_bytree': 1, '...",0.808097,4


# Submit

In [10]:
# data load 
df_test = pd.read_csv("C:\\Users\\icecr\\OneDrive\\CSV_file\\titanic\\test.csv")
#df_test = pd.read_csv("C:\\Users\\82103\\OneDrive\\CSV_file\\titanic\\test.csv")  #for notebook

test_id = df_test.PassengerId
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [11]:
X_df_test = transform_features_2(df_test)
X_df_test = transform_outlier(df=X_df_test, column='Fare', weight=4.5)

X_df_test.info()

<Outliers>
highest_val : 137.7189
lowest_val : -98.3231
len : 23

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Pclass      418 non-null    int64  
 1   Age         418 non-null    float64
 2   SibSp       418 non-null    int64  
 3   Parch       418 non-null    int64  
 4   Fare        418 non-null    float64
 5   Sex_male    418 non-null    uint8  
 6   Cabin_B     418 non-null    uint8  
 7   Cabin_C     418 non-null    uint8  
 8   Cabin_D     418 non-null    uint8  
 9   Cabin_E     418 non-null    uint8  
 10  Cabin_F     418 non-null    uint8  
 11  Cabin_G     418 non-null    uint8  
 12  Cabin_N     418 non-null    uint8  
 13  Embarked_Q  418 non-null    uint8  
 14  Embarked_S  418 non-null    uint8  
dtypes: float64(2), int64(3), uint8(10)
memory usage: 20.5 KB


In [12]:
#y_hat = xgb_clf.predict(X_df_test)  #XGB_raw(not tuning)

y_hat = xgb_grid.predict(X_df_test)  #GridCV tuned
print(len(y_hat))

418


In [13]:
submission = pd.DataFrame({'PassengerId':test_id, 'Survived':y_hat})
submission.to_csv("c:\\Users\\icecr\\OneDrive\\CSV_file\\titanic\\Sub25.csv", index=False)

##Verify label & id
# print(y_hat)
# for  i,a  in  enumerate(y_hat):
#     print (str(i+892) + ',' + str(a))